In [135]:
import pandas as pd
import numpy as np
from googletrans import Translator
import time
from folium.plugins import HeatMap

"""
docker-compose -f docker-compose-LocalExecutor.yml up -d
docker-compose -f docker-compose-LocalExecutor.yml down
"""

In [127]:
ventas = pd.read_csv('./data/olist_closed_deals_dataset.csv') # Demian  check
clientes = pd.read_csv('./data/olist_customers_dataset.csv') # Demian   check
geoloc = pd.read_csv('./data/olist_geolocation_dataset.csv')# Demian   check
lideres = pd.read_csv('./data/olist_marketing_qualified_leads_dataset.csv') # Facundo
items = pd.read_csv('./data/olist_order_items_dataset.csv') # Facundo
pagos = pd.read_csv('./data/olist_order_payments_dataset.csv') # Facundo
reviews = pd.read_csv('./data/olist_order_reviews_dataset.csv') # Leo   check
venta = pd.read_csv('./data/olist_orders_dataset.csv') # Leo    check
productos = pd.read_csv('./data/olist_products_dataset.csv') # Demian   check
vendedores = pd.read_csv('./data/olist_sellers_dataset.csv') # Leo  check
produtrans = pd.read_csv('./data/product_category_name_translation.csv') # no se usó

In [128]:
vendedores

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


In [171]:
'''
Limpieza y nomalización del DataFrame close_deals
'''

# Carga del archivo
ventas = pd.read_csv('./data/olist_closed_deals_dataset.csv')

# Cambiando la columna de fecha de tipo object a tipo datetime
ventas['won_date'] = pd.to_datetime(ventas['won_date'])

# Reemplazando los valores 0.0 y 'unknown' a np.nan (en columnas numericas)
ventas['average_stock'] = ventas['average_stock'].replace('unknown',np.nan)
ventas['declared_monthly_revenue'] = ventas['declared_monthly_revenue'].replace(0.0,np.nan)

# Reemplazando los valores NaN a 'Sin Dato' (en columnas tipo object/string)
ventas['business_segment'] = ventas['business_segment'].replace(np.nan,'Sin Dato')
ventas['lead_type'] = ventas['lead_type'].replace(np.nan,'Sin Dato')
ventas['lead_behaviour_profile'] = ventas['lead_behaviour_profile'].replace(np.nan,'Sin Dato')
ventas['has_company'] = ventas['has_company'].replace(np.nan,'Sin Dato')
ventas['has_gtin'] = ventas['has_gtin'].replace(np.nan,'Sin Dato')
ventas['business_type'] = ventas['business_type'].replace(np.nan,'Sin Dato')
ventas['average_stock'] = ventas['average_stock'].replace(np.nan,'Sin Dato')

# Removiendo valores outlier de la columna de declaracion mensual (ver justificacion en la variable mensual, mas abajo)
ventas.loc[ventas['declared_monthly_revenue'] < 1000,'declared_monthly_revenue'] = np.nan
ventas.loc[ventas['declared_monthly_revenue'] > 500000,'declared_monthly_revenue'] = np.nan

'''
mensual = pd.read_csv('./data/olist_closed_deals_dataset.csv')
mensual['average_stock'] = mensual['average_stock'].replace('unknown',np.nan)
mensual['declared_monthly_revenue'] = mensual['declared_monthly_revenue'].replace(0.0,np.nan)
mensual = mensual[['declared_monthly_revenue','lead_behaviour_profile','lead_type']]
mensual.loc[mensual['lead_behaviour_profile'].isna(), 'lead_behaviour_profile'] = 'vacio'
mensual.loc[mensual['lead_type'].isna(), 'lead_type'] = 'vacio'
mensual = mensual.dropna()
mensual = mensual.sort_values(by=['declared_monthly_revenue'])
mensual
'''

"\nmensual = pd.read_csv('./data/olist_closed_deals_dataset.csv')\nmensual['average_stock'] = mensual['average_stock'].replace('unknown',np.nan)\nmensual['declared_monthly_revenue'] = mensual['declared_monthly_revenue'].replace(0.0,np.nan)\nmensual = mensual[['declared_monthly_revenue','lead_behaviour_profile','lead_type']]\nmensual.loc[mensual['lead_behaviour_profile'].isna(), 'lead_behaviour_profile'] = 'vacio'\nmensual.loc[mensual['lead_type'].isna(), 'lead_type'] = 'vacio'\nmensual = mensual.dropna()\nmensual = mensual.sort_values(by=['declared_monthly_revenue'])\nmensual\n"

In [5]:
'''
Limpieza y normalizacion del DataFrame customers
'''

# Carga del archivo
clientes = pd.read_csv('./data/olist_customers_dataset.csv')

# Creando una serie con lo customer_unique_id repetidos
repetidos = []
for i in clientes[clientes['customer_unique_id'].duplicated()]['customer_unique_id'].unique():
    if clientes[clientes['customer_unique_id'] == i].shape[0] > 1:
        repetidos.append(i)
repetidos = pd.Series(repetidos)

# Creando una serie con los zip codes repetidos dentro de la serie de customer_unique_id repetidos
# en resumen: los registros de clientes con todos los datos duplicados, son 250
zip_repe = []
for i in repetidos:
     if (clientes[clientes['customer_unique_id'] == i]['customer_zip_code_prefix'].unique().shape[0]) == 1:
        zip_repe.append(i)
zip_repe = pd.Series(zip_repe)

# Guardo aparte los clientes duplicados
guarda = clientes[clientes['customer_unique_id'].isin(zip_repe)].sort_values(by='customer_zip_code_prefix')

# Elimino de la tabla original todos los duplicados
clientes = clientes.drop(clientes[clientes['customer_unique_id'].isin(zip_repe)].index)

# De la tabla guardada remuevo los duplicados, quedandome con el dato mas antiguo
guarda = guarda.drop_duplicates(subset='customer_unique_id',keep='first')

# Uno las dos tablas quedando limpia de clientes con domicilio repetido
clientes = pd.concat([clientes,guarda])

# Reseteo el index
clientes.index = range(clientes.shape[0])

# Queda solucionar el problema del dataframe ordenes que está relacionado a customer_id's que fueron borrados
# Abajo se carga una tabla con las dos columnas para ayudar a hacer el merge
relacion = pd.read_csv('./data/olist_customers_dataset.csv')
relacion = relacion[['customer_id','customer_unique_id']]
'''
Cabe destacar que en el customer_city final hay un dato menos, queda normalizado planaltina de goias a planaltina
probar lo siguiente:
print(clientes['customer_city'].unique().shape)
print(relacion['customer_city'].unique().shape)
relacion[relacion['customer_city']=='planaltina de goias']
clientes[clientes['customer_unique_id']=='7d3d94b4740895a17760e976797f9f0e']
relacion[relacion['customer_city']=='planaltina']
No habria que aplicar otra corrección
'''

"\nCabe destacar que en el customer_city final hay un dato menos, queda normalizado planaltina de goias a planaltina\nprobar lo siguiente:\nprint(clientes['customer_city'].unique().shape)\nprint(relacion['customer_city'].unique().shape)\nrelacion[relacion['customer_city']=='planaltina de goias']\nclientes[clientes['customer_unique_id']=='7d3d94b4740895a17760e976797f9f0e']\nrelacion[relacion['customer_city']=='planaltina']\nNo habria que aplicar otra corrección\n"

In [188]:
'''
Limpieza y normalizacion del DataFrame products
'''

# Carga del archivo
productos = pd.read_csv('./data/olist_products_dataset.csv')

# Reemplazo del _ por espacio para mejor visualizacion del dato
productos['product_category_name'] = productos['product_category_name'].str.replace('_',' ')

# Reemplazo de los nan por el string 'Sin registrar'
#productos['product_category_name'] = productos['product_category_name'].replace(np.nan,'Sin registrar')

# Traduciendo la categoria con google translate
prod = pd.DataFrame(productos['product_category_name'].unique())
prod = prod.rename(columns={0:'product_category_name'})
prod['espanol'] = ''
# Traduciendo
traductor = Translator(service_urls=['translate.google.com.ar'])
for i in range(0,len(prod['product_category_name'])):
    traduccion = traductor.translate(prod['product_category_name'][i], src='pt', dest='es')
    prod['espanol'][i] = traduccion.text
# Uniendo la lista traducida y reacomodando
productos = pd.merge(productos, prod, on ='product_category_name', how = 'left')
productos = productos[['product_id','product_category_name','espanol','product_name_lenght','product_description_lenght',
                    'product_photos_qty','product_weight_g','product_length_cm','product_height_cm','product_width_cm']]

# Colocando primera letra mayúscula
productos['product_category_name'] = productos['product_category_name'].str.capitalize()
productos['espanol'] = productos['espanol'].str.capitalize()

# Reemplazando los valores NaN a 'Sin Dato' (en columnas tipo object/string)
productos['product_category_name'] = productos['product_category_name'].replace(np.nan,'Sin Dato')
productos['espanol'] = productos['espanol'].replace('Yaya','Sin Dato')

In [123]:
'''
Limpieza y normalizacion del DataFrame geolocation
'''

# Se carga el df original y una variable con los zip unicos
geoloc = pd.read_csv('./data/olist_geolocation_dataset.csv')
zipuni = geoloc['geolocation_zip_code_prefix'].unique()

# Se crea otro df con las mismas columnas (distinto nombre) para cargar los datos limpios
geoloc_limpia = pd.DataFrame(columns=['zip','lat','lon','ciudad','estado'])

# Hay 1 millon de datos en la tabla oribinal y solo 19 mil zip unicos
# Se procede a calcular la media de la ubicacion por cada zip
# Se agregan al df vacio. No se pierde ningun zip, ni se repite, lo cual serviría como PK
ind = 0
for i in range(0,len(zipuni)):
    zip = zipuni[i]
    filtro = geoloc[geoloc['geolocation_zip_code_prefix'] == zip]
    lat = filtro['geolocation_lat'].mean()
    lon = filtro['geolocation_lng'].mean()
    ciudad = filtro.iloc[0]['geolocation_city']
    estado = filtro.iloc[0]['geolocation_state']
    agrega = pd.DataFrame({'zip':zip,'lat':lat,'lon':lon,'ciudad':ciudad,'estado':estado},index=[ind])
    geoloc_limpia = pd.concat([geoloc_limpia,agrega])
    ind+=1

# Se detectan 15 outliers de ubicaciones fuera del pais buscando de esta forma
#limpia = geoloc_limpia[geoloc_limpia['lat'] > -15]
#limpia = limpia[limpia['lon'] > - 30]

# En base a los nombres de las ciudades se les asigna su ubicacion correcta
# (se corrobora con las coord de otras ciudades del mismo estado)
geoloc_limpia.loc[geoloc_limpia['zip'] == 18243,'lat'] = -23.54584
geoloc_limpia.loc[geoloc_limpia['zip'] == 18243,'lon'] = -48.30460
geoloc_limpia.loc[geoloc_limpia['zip'] == 46560,'lat'] = -12.98470
geoloc_limpia.loc[geoloc_limpia['zip'] == 46560,'lon'] = -42.22145
geoloc_limpia.loc[geoloc_limpia['zip'] == 57319,'lat'] = -9.73376
geoloc_limpia.loc[geoloc_limpia['zip'] == 57319,'lon'] = -36.73074
geoloc_limpia.loc[geoloc_limpia['zip'] == 58441,'lat'] = -7.18219
geoloc_limpia.loc[geoloc_limpia['zip'] == 58441,'lon'] = -35.98457
geoloc_limpia.loc[geoloc_limpia['zip'] == 68275,'lat'] = -1.46698
geoloc_limpia.loc[geoloc_limpia['zip'] == 68275,'lon'] = -56.37919
geoloc_limpia.loc[geoloc_limpia['zip'] == 68379,'lat'] = -8.31638
geoloc_limpia.loc[geoloc_limpia['zip'] == 68379,'lon'] = -55.10078
geoloc_limpia.loc[geoloc_limpia['zip'] == 78131,'lat'] = -15.64872
geoloc_limpia.loc[geoloc_limpia['zip'] == 78131,'lon'] = -56.14690
geoloc_limpia.loc[geoloc_limpia['zip'] == 83252,'lat'] = -25.53323
geoloc_limpia.loc[geoloc_limpia['zip'] == 83252,'lon'] = -48.50862
geoloc_limpia.loc[geoloc_limpia['zip'] == 95130,'lat'] = -29.24143
geoloc_limpia.loc[geoloc_limpia['zip'] == 95130,'lon'] = -51.01992
geoloc_limpia.loc[geoloc_limpia['zip'] == 68447,'lat'] = -1.50876
geoloc_limpia.loc[geoloc_limpia['zip'] == 68447,'lon'] = -48.61568
geoloc_limpia.loc[geoloc_limpia['zip'] == 11990,'lat'] = -25.01060
geoloc_limpia.loc[geoloc_limpia['zip'] == 11990,'lon'] = -47.92986
geoloc_limpia.loc[geoloc_limpia['zip'] == 15855,'lat'] = -21.17783
geoloc_limpia.loc[geoloc_limpia['zip'] == 15855,'lon'] = -49.19197
geoloc_limpia.loc[geoloc_limpia['zip'] == 28155,'lat'] = -21.97254 # hay 5 santa maria distintos en RJ
geoloc_limpia.loc[geoloc_limpia['zip'] == 28155,'lon'] = -42.00692 # se eligió uno
geoloc_limpia.loc[geoloc_limpia['zip'] == 47310,'lat'] = -9.29524
geoloc_limpia.loc[geoloc_limpia['zip'] == 47310,'lon'] = -40.81975
geoloc_limpia.loc[geoloc_limpia['zip'] == 29654,'lat'] = -19.82352
geoloc_limpia.loc[geoloc_limpia['zip'] == 29654,'lon'] = -40.65459

In [ ]:
import folium
from folium.plugins import MarkerCluster

x = pd.Series(geoloc_limpia['lat'])
y = pd.Series(geoloc_limpia['lon'])

mapa = folium.Map(location = [x.mean(),y.mean()], zoom_start=4, control_scale=True)

mc = MarkerCluster()
for row in geoloc_limpia.values:
    mc.add_child(folium.Marker(location=[row[1],row[2]]))
mapa.add_child(mc)

mapa

In [ ]:
heatmap = geoloc_limpia[['lat','lon']]
heatmap['heat'] = pd.Series(0.2,index=range(0,19015))

mapa = folium.Map(location = [geoloc_limpia['lat'].mean(),geoloc_limpia['lon'].mean()], zoom_start=4, control_scale=True)

HeatMap(heatmap,radius=5,blur=5).add_to(mapa)
#mapa.save('heatmap.html')
mapa

In [199]:
'''
Limpieza y normalizacion del DataFrame reviews
'''

# Cargando el archivo
reviews = pd.read_csv('./data/olist_order_reviews_dataset.csv')

# Reemplazo nulos en comentario por 'Sin Dato'
reviews['review_comment_message'] = reviews['review_comment_message'].replace(np.nan,'Sin Dato')
reviews['review_comment_title'] = reviews['review_comment_title'].replace(np.nan,'Sin Dato')

In [ ]:
'''
Limpieza y normalizacion del DataFrame sellers
'''

# Cargando el archivo
vendedores = pd.read_csv('./data/olist_sellers_dataset.csv')



In [204]:
'''
Limpieza y normalizacion del DataFrame orders
'''

# Cargando el archivo
venta = pd.read_csv('./data/olist_orders_dataset.csv') 





In [ ]:
''' ingestamos en un dataframe los clientes calificados de marketing a partir 
de las columnas landing_page_id y origen obtenemos dos tablas con valores unicos para la normalizacion de estos campos'''

df10= pd.read_csv('./data/olist_marketing_qualified_leads_dataset.csv')
#lideres = pd.read_csv('./data/olist_marketing_qualified_leads_dataset.csv')

# Creamos un df para normalizar los id de pagina alfanumericos

df20= pd.DataFrame(df10['landing_page_id'].unique())
df20.columns =['landing_page_id']
df20.columns.name='IdPage'
df20['IdPages']=np.arange(1,len(df20)+1)
df20.to_csv("~/ip_files/olist_landing_page_id_Normalizada.csv")


'''Creo una columna Idlanding para sustituir el campo landing_page_id alfanumerico por un valor numerico'''
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
landing = df20.landing_page_id.value_counts().index
landing_page_unique = df10.landing_page_id.unique()
normalized = []
def get_matches(query,choices):
    for i in query:
        tuple = process.extractOne(i,choices)
        normalized.append(tuple[0])
    return normalized
landing_page_correg = get_matches(landing_page_unique, landing)
mydict = {landing_page_unique[i]: df20[df20['landing_page_id']==landing_page_correg[i]]['IdPages'] for i in range(0,495)}
df10 = df10.convert_dtypes()
df10["Idlanding"] = df10["landing_page_id"].map(mydict)
df31 =df10['Idlanding']
df32=list(df31.astype(str))
df31=np.arange(len(df31))
lindice=[]
for i in df31 :
    cadena=df32[i].split("\n")
    indice=cadena[0].split(" ")
    lindice.append(indice[4])
df10['IdLandingN']=lindice
df10.drop(['landing_page_id','Idlanding'], axis=1, inplace=True)
df10.rename(columns={'IdLandingN': 'landing_page_id'}, inplace=True)

'''Creo una columna IdOrigin para sustituir el campo Origin alfanumerico por un valor numerico'''
df10['origin'].fillna('unknown',inplace=True)
df21=pd.DataFrame(df10['origin'].unique())
df21.columns =['Origin']
df21.columns.name='IdOrigin'
df21['IdPages']=np.arange(1,len(df21)+1)
df21.to_csv("~/ip_files/olist_IdOrigin_Normalizada.csv")

#comparo y sustituyo
Origin = df21.Origin.value_counts().index
Origin_unique = df10.origin.unique()
normalized = []
def get_matches(query,choices):
    for i in query:
        tuple = process.extractOne(i,choices)
        normalized.append(tuple[0])
    return normalized
Origin_correg = get_matches(Origin_unique, Origin)
mydict = {Origin_unique[i]: df21[df21['Origin']==Origin_correg[i]]['IdPages'] for i in range(0,10)}
df10 = df10.convert_dtypes()
df10["IdOrigin"] = df10["origin"].map(mydict)
df33 =df10['IdOrigin']
df34=list(df33.astype(str))
df33=np.arange(len(df33))
lindice=[]
for i in df33 :
    cadena=df34[i].split("\n")
    indice=cadena[0].split(" ")
    lindice.append(indice[4])
df10['IdOriginN']=lindice
df10.drop(['origin','IdOrigin'], axis=1, inplace=True)
df10.rename(columns={'IdOriginN': 'IdOrigin'}, inplace=True)

'''creacion de tabla con valores unicos para normalización'''
df35= pd.DataFrame(df10['mql_id'].unique())
df35.columns =['mql_id']
df35.columns.name='Imql_id'
df35['mql_idN']=np.arange(1,len(df35)+1)
df35.to_csv("~/ip_files/olist_mql_idN_Normalizada.csv")

'''Creo una columna mql_idN para sustituir el campo mql_id alfanumerico por un valor numerico'''
df10['mql_idN']=np.arange(1,len(df10)+1)
df10.drop(['mql_id'], axis=1, inplace=True)
df10.rename(columns={'mql_idN': 'mql_id'}, inplace=True)

'''Cambiar el tipo de datos a las columnas'''
df10['first_contact_date'] = pd.to_datetime(df10['first_contact_date'])
df10['landing_page_id'] = pd.to_numeric(df10['landing_page_id'])
df10['IdOrigin'] = pd.to_numeric(df10['IdOrigin'])
df10['mql_id'] = pd.to_numeric(df10['mql_id'])



In [8]:
venta = venta[['order_id','customer_id','order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date']]
venta = venta.rename(columns={'order_id':'IdVenta','customer_id':'IdCliente','order_purchase_timestamp':'fecha_compra','order_approved_at':'fecha_aprobado','order_delivered_carrier_date':'fecha_envio','order_delivered_customer_date':'fecha_arribo'})
venta.head()

,IdVenta,IdCliente,fecha_compra,fecha_aprobado,fecha_envio,fecha_arribo
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02


In [9]:


items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
